In [ ]:
from Layer.CSP_MB_Layers import *
from utils.DataGenerator import load_data
from model import *
from models.config import CustomizeSmall, CustomizeLarge, MobileNetLarge, MobileNetSamll, EfficientNetB0
from utils.Flops import get_flops

In [ ]:
# 模型參數設定
# MobileNetV3Large、 MobileNetV3Small、 EfficientNetB0、 CustomizeLarge 、CustomizeSmall
backbone = 'CustomizeSmall'
# SE CBAM CA
SE_CBAM_CA = 'SE'
# Adam RMSprop CLR
LR_mode = 'Adam'
unfrozen = []

# 圖片大小
input_shape = None
# 類別數
num_classes = 0

# 訓練參數
BATCH_SIZE = 128
epoch_1 = 150
epoch_2 = 500
Dropout_rate = 0.5

# 100 Bird Species  or  325 Bird Species  or  cifar100 or cifar10
Dataset =["100 Bird Species", "325 Bird Species", "cifar100", "cifar10"]
Dataset = Dataset[2]

# Load Dataset

In [ ]:
if Dataset == "325 Bird Species":
    dir = "./Dataset/325 Bird Species/"
    input_shape = (224,224,3)
    num_classes = 325
    trainset, valset, testset = load_data(Dataset, dir, input_shape, BATCH_SIZE)
    
elif Dataset == "100 Bird Species":
    dir = "./Dataset/100 Bird Species/"
    input_shape = (224,224,3)
    num_classes = 100
    trainset, valset, testset = load_data(Dataset, dir, input_shape, BATCH_SIZE)
    
elif Dataset == "cifar100":
    input_shape = (32,32,3)
    num_classes = 100
    datagen_train, datagen_val, x_train, y_train, x_val, y_val, x_test, y_test = load_data(Dataset, '', input_shape, num_classes)
    
elif Dataset == "cifar10":
    input_shape = (32,32,3)
    num_classes = 10
    datagen_train, datagen_val, x_train, y_train, x_val, y_val, x_test, y_test = load_data(Dataset, '', input_shape, num_classes)

In [ ]:
if  backbone == "CustomizeLarge":
    Specification, unfrozen = CustomizeLarge(Dataset)
elif backbone == "CustomizeSmall":
    Specification, unfrozen = CustomizeSmall(Dataset)
elif backbone == 'MobileNetV3Large':
    Specification, unfrozen = MobileNetLarge(Dataset)
elif backbone == 'MobileNetV3Small':
    Specification, unfrozen = MobileNetSamll(Dataset)


base_model = build_base_model(input_shape, SE_CBAM_CA, Specification)
# base_model.summary()

# Build Model

In [ ]:
predtions = predictions_head(base_model, num_classes, Dropout_rate)
model = build_model(base_model, predtions)
model.summary()
print("The FLOPs is:{}".format(get_flops(model)) ,flush=True)

# Compile

In [ ]:
INIT_LR = 0.00025
MAX_LR = 0.001
if Dataset == "325 Bird Species" or Dataset == "100 Bird Species":
    steps_per_epoch = trainset.samples // BATCH_SIZE
elif Dataset == "cifar10" or Dataset == "cifar100":
    steps_per_epoch = len(x_train) // BATCH_SIZE

In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy

optimizer = optimizer_set(LR_mode=LR_mode,
                          INIT_LR=INIT_LR,
                          MAX_LR=MAX_LR,
                          steps_per_epoch=steps_per_epoch)

loss = CategoricalCrossentropy(label_smoothing=0.1)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Callback

In [ ]:
from IPython.display import clear_output
from utils.gradcam import GradCAM, overlay_gradCAM
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'grad-cam']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

def show_predictions(dataset=None, num=1):
    gradCAM = GradCAM(model=model, layerName="Conv3hardswish")
    
    if Dataset == "325 Bird Species" or Dataset == "100 Bird Species":
        img = cv2.imread('Dataset/325 Bird Species/train\YELLOW CACIQUE/010.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        upsample_size = (img.shape[0],img.shape[1])
        
        pred_img = tf.image.resize(img,(224,224))
        pred_img = tf.reshape(pred_img, (1,224,224,3))
        pred_img = tf.cast(pred_img, dtype=tf.float32)/255
        
    elif Dataset == "cifar10" or Dataset == "cifar100":
        img = cv2.imread('Dataset/cifar10/train/13.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        upsample_size = (img.shape[0],img.shape[1])
        
        pred_img = tf.image.resize(img,(32,32))
        pred_img = tf.reshape(pred_img, (1,32,32,3))
        pred_img = tf.cast(pred_img, dtype=tf.float32)/255
    
    pred = model.predict(pred_img)
    classIdx = pred.argmax()
    cam3 = gradCAM.compute_heatmap(image=pred_img, classIdx=classIdx, upsample_size=upsample_size)
    gradcam = overlay_gradCAM(img, cam3)
    gradcam = cv2.cvtColor(gradcam, cv2.COLOR_BGR2RGB)
    display([img, gradcam])
    
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))


In [ ]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping,ReduceLROnPlateau

checkpoint_path = 'weights/{0}/{1}_{2}_{3}/'.format(backbone, SE_CBAM_CA, LR_mode, Dataset) + "/cp-{epoch:04d}.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1, 
                             save_weights_only=True,
                             save_best_only=True, 
                             mode='max',
                             save_freq='epoch')

tensorboard = TensorBoard(log_dir="logs/{0}/{1}_{2}_{3}/".format(backbone, SE_CBAM_CA, LR_mode, Dataset),
                          histogram_freq=0,
                          write_graph=True,
                          write_images=False,
                          update_freq="epoch",
                          profile_batch=2,
                          embeddings_freq=0,
                          embeddings_metadata=None,)

# Training Model

- 如果選擇的是自定義模型，則需要從頭開始訓練整個模型
- 若選擇其他模型，則直接轉至Fine-tuning

## Training-1

In [ ]:
if Dataset == "325 Bird Species" or Dataset == "100 Bird Species":
    history_ft = model.fit(trainset,
                        epochs=epoch_1,
                        validation_data=valset,
                        callbacks = [tensorboard, checkpoint, DisplayCallback()],
                        # verbose=0
                        )
elif Dataset == "cifar10" or Dataset == "cifar100":
    history_ft = model.fit(datagen_train.flow(x_train, y_train, batch_size=BATCH_SIZE),
                            epochs=epoch_1,
                            validation_data=datagen_val.flow(x_val, y_val, batch_size=BATCH_SIZE),
                            callbacks = [tensorboard, checkpoint, DisplayCallback()],
                            # verbose=0
                            )

# Fine tuned model

## 設定unfrozen的Layer

In [ ]:
setup_to_transfer_learning(base_model, unfrozen=unfrozen)
base_model.summary()

# Training-2

In [ ]:
# if Dataset == "325 Bird Species" or Dataset == "100 Bird Species":
#     history_ft = model.fit(trainset,
#                             epochs=epoch_1 + epoch_2,
#                             initial_epoch=epoch_1,
#                             validation_data=valset,
#                             callbacks=callbacks)
# elif Dataset == "cifar10" or Dataset == "cifar100":
#     history_ft = model.fit(datagen_train.flow(x_train, y_train, batch_size=BATCH_SIZE),
#                             epochs=epoch_1 + epoch_2,
#                             initial_epoch=epoch_1,
#                             validation_data=datagen_val.flow(x_val, y_val, batch_size=BATCH_SIZE),
#                             callbacks=callbacks,
#                             )

# SAVE Model

In [ ]:
import os
path = './weights/{0}/{1}_{2}_{3}'.format(backbone, SE_CBAM_CA, LR_mode, Dataset)
model_num = ''
print('Fine模型列表：')
for a in os.listdir(path):
    print(a)
    model_num = a
print('最新的model為：',model_num)
model_num = a[:-6]

In [ ]:
model.load_weights('./weights/{0}/{1}_{2}_{3}/{4}'.format(backbone,SE_CBAM_CA,LR_mode, Dataset, model_num))

In [ ]:
model.save('./weights/{0}/{1}_{2}_{3}/best_model'.format(backbone, SE_CBAM_CA, LR_mode, Dataset, model_num))

In [ ]:
# path to the SavedModel directory
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_dir = './weights/{0}/{1}_{2}_{3}/tflite/'.format(backbone, SE_CBAM_CA, LR_mode, Dataset)
if not os.path.isdir(tflite_dir):
    os.makedirs(tflite_dir)

# Save the model.
with open(tflite_dir + "model.tflite", 'wb') as f:
    f.write(tflite_model)